In [4]:
import os
os.environ["TFHUB_CACHE_DIR"] = r'C:\Users\HP\Documents\PYTHON CODE\GitHub\Artificial Intelligent Models\BERT_MODEL\saved_model'

In [1]:
import tensorflow_hub as hub 
import tensorflow_text as txt 

In [2]:
preprocessed_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
bert_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'

In [5]:
preprocessor = hub.KerasLayer(preprocessed_url)
bert= hub.KerasLayer(bert_url)

In [6]:
sample_text = ['this is a good boy', 'this is a good girl'] 
pre_obj = preprocessor(sample_text)
pre_obj.keys()

dict_keys(['input_type_ids', 'input_mask', 'input_word_ids'])

In [30]:
pre_obj['input_word_ids']

<tf.Tensor: shape=(2, 128), dtype=int32, numpy=
array([[ 101, 2023, 2003, 1037, 2204, 2879,  102,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0],
       [ 101, 2023, 2003, 1037, 2204, 2611,  102,    0,    0,    0,    0,
           0,

In [32]:
bert_obj = bert(pre_obj)
bert_obj.keys()

dict_keys(['pooled_output', 'encoder_outputs', 'sequence_output', 'default'])

In [35]:
len(bert_obj['encoder_outputs'])

12

In [38]:
bert_obj['pooled_output']

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[-0.7622522 , -0.15072162,  0.63359857, ...,  0.44831866,
        -0.48790509,  0.76934254],
       [-0.75591516, -0.13169983,  0.6928331 , ...,  0.5635903 ,
        -0.4705333 ,  0.7703518 ]], dtype=float32)>

# load dataset 

In [7]:
import pandas as pd 

df = pd.read_csv('spam.csv', encoding='latin-1')
df.rename(columns={'v1':'target','v2':'text'},inplace=True)
df.drop(columns= df.columns[2:], inplace=True)
df

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [8]:
from sklearn.preprocessing import LabelEncoder 
from sklearn import model_selection

lb = LabelEncoder()
df['target'] = lb.fit_transform(df.label)
df

,label,text,target
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1
5568,ham,Will Ì_ b going to esplanade fr home?,0
5569,ham,"Pity, * was in mood for that. So...any other s...",0
5570,ham,The guy did some bitching but I acted like i'd...,0


In [9]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(df['text'], df['target'])

In [10]:
import tensorflow as tf 

# loading input layer with bert preprocessor 
input_layer = tf.keras.layers.Input(shape=() , dtype=tf.string, name='text')

# bert layer
encoder_input = preprocessor(input_layer)
encoder = bert(encoder_input)

# keras layer 
lo = tf.keras.layers.Dropout(0.1, name='droplayer')(encoder['pooled_output'])
fo  = tf.keras.layers.Dense(1, activation='sigmoid', name='denselayer')(lo)

# model building
model = tf.keras.Model(inputs = [input_layer], outputs=[fo])

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_type_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128)}                                                  

In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics='accuracy')
model.fit(X_train , y_train, epochs=2)

Epoch 1/2
131/131 [==============================] - 1499s 11s/step - loss: 0.3192 - accuracy: 0.8655
Epoch 2/2
131/131 [==============================] - 1633s 12s/step - loss: 0.2370 - accuracy: 0.8978


In [13]:
model.evaluate(X_test, y_test)

44/44 [==============================] - 432s 10s/step - loss: 0.1979 - accuracy: 0.9275


[0.19791105389595032, 0.9274946451187134]

In [15]:
model.save('spam_bert_model.h5')